# 题目

> 为搜索引擎设计一个搜索自动补全系统。用户会输入一条语句（最少包含一个字母，以特殊字符 '#' 结尾）。  
给定一个字符串数组 sentences 和一个整数数组 times ，长度都为 n ，其中 sentences[i] 是之前输入的句子， times[i] 是该句子输入的相应次数。对于除 ‘#’ 以外的每个输入字符，返回前 3 个历史热门句子，这些句子的前缀与已经输入的句子的部分相同。  
下面是详细规则：  
- 一条句子的热度定义为历史上用户输入这个句子的总次数。
- 返回前 3 的句子需要按照热度从高到低排序（第一个是最热门的）。如果有多条热度相同的句子，请按照 ASCII 码的顺序输出（ASCII 码越小排名越前）。
- 如果满足条件的句子个数少于 3 ，将它们全部输出。
- 如果输入了特殊字符，意味着句子结束了，请返回一个空集合。

> 实现 AutocompleteSystem 类：  
1、AutocompleteSystem(String[] sentences, int[] times): 使用数组 sentences 和 times 对对象进行初始化。  
2、List<String> input(char c) 表示用户输入了字符 c 。
- 如果 c == '#' ，则返回空数组 [] ，并将输入的语句存储在系统中。
- 返回前 3 个历史热门句子，这些句子的前缀与已经输入的句子的部分相同。如果少于 3 个匹配项，则全部返回。

# 方法一：字典树

> 见代码。

## 复杂度

- 时间复杂度：初始化类的时间复杂度为 $O(k×l)$，其中 $k$ 是句子的平均长度， $l$ 是句子数量； input() 方法的时间复杂度为 $O\big(p+q+mlog(m)\big)$ ，其中 $p$ 表示当前已输入句子 $cursent$ 的长度， $q$ 表示单词查找树中的节点数量， $m$ 为包含前缀的句子的长度。

> 对长度为 $m$ 的 list 排序需要时间 $O\big(mlog(m)\big)$ 。

## 代码

In [1]:
class AutocompleteSystem:

    def __init__(self, sentences, times):
        # 属性1：字典树
        self.trie = {}
        # 将列表sentences中的每句话存储到字典树中
        for i,sentence in enumerate(sentences):
            node = self.trie
            for c in sentence:
                if c not in node:
                    node[c] = {}
                node = node[c]
            node["#"] = times[i]  # 子节点的key'#'对应的value的用来保存这个句子出现的次数
        # 属性2：字典树的根节点
        self.node = self.trie
        # 属性3：目前为止的前缀
        self.prefix = []

    # 输入字符c，返回包含当前前缀的热度最高的三句话
    def input(self, c):
        if c == "#":
            # 如果当前节点中包含名为'#'的key
            if "#" in self.node:
                # 这句话的出现次数+1
                self.node["#"] += 1
            # 否则说明这句话第一次出现
            else:
                self.node["#"] = 1
            # 用户已经输入完了一句话，因此将prefix和node重置
            self.prefix = []
            self.node = self.trie
            return []

        # 如果c不是'#'，且c不在当前子节点的字典里
        if c not in self.node:
            # 创建一个用于表示这个c的子节点
            self.node[c] = {}
        # 移动到这个子节点
        self.node = self.node[c]
        # 将当前的字符c加入前缀列表
        self.prefix.append(c)
        
        ans = []

        # 对代表c的子节点进行dfs
        # 这个子节点的每一个key都表示c的下一个可能的字符
        self.dfs(self.node, self.prefix, ans)

        # 按出现次数降序排列，按ASCII升序排列
        ans.sort(key=lambda x:(-x[0],x[1]))
        # 取前3
        return [x[1] for x in ans][:3]

    
    def dfs(self, node, prefix, ans):
        # 遍历当前子节点字典的每一个key
        for k in node:
            # 如果k不是'#'，说明这句话没有结束
            if k != "#":
                # 将当前字符k加入prefix中
                prefix.append(k)
                # 递归地进行dfs，直到搜索到'#'才会回溯
                self.dfs(node[k], prefix, ans)
                # 所有新添加的k都会被pop()删除，因此prefix会恢复到最开始输入时的状态
                prefix.pop()
            # 如果k是'#'，说明在列表sentences中找到了一个包含当前前缀的句子，将其添加到ans
            else:
                ans.append([node["#"],"".join(prefix)])  # 出现次数，句子

#### 测试一

In [2]:
sentences = ["i love you", "island", "iroman", "i love leetcode"]
times = [5, 3, 2, 2]

autocompleteSystem = AutocompleteSystem(sentences, times)
autocompleteSystem.input('i')

['i love you', 'island', 'i love leetcode']

In [3]:
autocompleteSystem.input(' ')

['i love you', 'i love leetcode']

In [4]:
autocompleteSystem.input('a')

[]

In [5]:
autocompleteSystem.input('#')

[]